Questions about the dataset:
1. How has the total number of small firms and establishments changed over time? What does this reveal about the overall trend of small business growth in the US?
2. Which sectors show the highest new firm entry and exit rates over time?
3. How do new establishment entry rates and exit rates compare over time? What does this suggest about the ease of entering versus surviving in the market?
4. How does job creation from new firms compare to job destruction from firm deaths, and what does this balance reveal about the sustainability of small business employment?
5. How does the business reallocation rate reflect broader shifts in the US small business trend? Does it have anything to do with movement from traditional market to new digital operations?

In [2]:
import numpy as np
import pandas as pd
import duckdb

In [3]:
s=pd.read_csv('Sector_Age_Size.csv')

In [4]:
id_cols=['year','sector','fage','fsize']
data_cols=[c for c in s.columns if c not in id_cols]
s.replace(['D','X', 'N'], pd.NA,  inplace=True)
s.dropna(subset=data_cols, how='all')

,year,sector,fage,fsize,firms,estabs,emp,denom,estabs_entry,estabs_entry_rate,...,job_destruction_deaths,job_destruction_continuers,job_destruction_rate_deaths,job_destruction_rate,net_job_creation,net_job_creation_rate,reallocation_rate,firmdeath_firms,firmdeath_estabs,firmdeath_emp
0,1978,11,a) 0,a) 1 to 4,2969,2969,8050,4025,2969,200.000,...,<NA>,<NA>,<NA>,<NA>,8050,200.000,<NA>,<NA>,<NA>,<NA>
1,1978,11,a) 0,b) 5 to 9,213,215,2780,1390,215,200.000,...,<NA>,<NA>,<NA>,<NA>,2780,200.000,<NA>,<NA>,<NA>,<NA>
2,1978,11,a) 0,c) 10 to 19,66,66,1786,893,66,200.000,...,<NA>,<NA>,<NA>,<NA>,1786,200.000,<NA>,<NA>,<NA>,<NA>
3,1978,11,a) 0,d) 20 to 99,32,32,2130,1065,32,200.000,...,<NA>,<NA>,<NA>,<NA>,2130,200.000,<NA>,<NA>,<NA>,<NA>
5,1978,11,a) 0,f) 500 to 999,0,0,0,0,0,<NA>,...,<NA>,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104875,2023,81,l) Left Censored,f) 500 to 999,276,2820,83597,81063,102,3.644,...,380,3135,0.469,4.336,5064,6.247,8.672,0,0,0
104876,2023,81,l) Left Censored,g) 1000 to 2499,225,3927,100553,97841,137,3.517,...,938,4073,0.959,5.122,5608,5.732,10.243,0,0,0
104877,2023,81,l) Left Censored,h) 2500 to 4999,118,1336,61084,58076,70,5.249,...,1086,2855,1.870,6.786,5644,9.718,13.572,0,0,0
104878,2023,81,l) Left Censored,i) 5000 to 9999,106,3641,86071,82953,408,11.650,...,775,2800,0.934,4.310,5681,6.848,8.619,0,0,0


In [10]:
firms_num=duckdb.sql("""SELECT year, SUM(CAST(firms AS DOUBLE)) AS total_firms
                            FROM s
                            GROUP BY year
                            ORDER BY year""").df()
print(firms_num)

    year  total_firms
0   1978    3640560.0
1   1979    3768015.0
2   1980    3814944.0
3   1981    3844382.0
4   1982    3823728.0
5   1983    3913815.0
6   1984    4078305.0
7   1985    4150391.0
8   1986    4246245.0
9   1987    4314562.0
10  1988    4400361.0
11  1989    4472245.0
12  1990    4535648.0
13  1991    4518372.0
14  1992    4512368.0
15  1993    4587600.0
16  1994    4668766.0
17  1995    4764867.0
18  1996    4810734.0
19  1997    4860478.0
20  1998    4932704.0
21  1999    4961353.0
22  2000    4981078.0
23  2001    4992018.0
24  2002    5018902.0
25  2003    5072006.0
26  2004    5160827.0
27  2005    5242673.0
28  2006    5337493.0
29  2007    5375178.0
30  2008    5310711.0
31  2009    5165537.0
32  2010    5093903.0
33  2011    5070530.0
34  2012    5117551.0
35  2013    5141822.0
36  2014    5177981.0
37  2015    5225684.0
38  2016    5291337.0
39  2017    5326629.0
40  2018    5356555.0
41  2019    5406194.0
42  2020    5410887.0
43  2021    5468780.0
44  2022  

In [27]:
sector_entry_exit=duckdb.sql("""SELECT sector, year, SUM(CAST(estabs_entry AS DOUBLE)) AS entry, SUM(CAST(estabs_exit AS DOUBLE)) AS exit
                            FROM s
                            GROUP BY sector, year
                            ORDER BY sector, year""").df()
print(sector_entry_exit)

    sector  year    entry     exit
0       11  1978   4005.0   3718.0
1       11  1979   3598.0   3690.0
2       11  1980   3259.0   3386.0
3       11  1981   2747.0   3479.0
4       11  1982   2597.0   3444.0
..     ...   ...      ...      ...
869     81  2019  56544.0  51828.0
870     81  2020  59867.0  61807.0
871     81  2021  67894.0  70884.0
872     81  2022  77455.0  59099.0
873     81  2023  73337.0  59829.0

[874 rows x 4 columns]


In [28]:
entry_exit=duckdb.sql("""SELECT year, SUM(CAST(estabs_entry AS DOUBLE)) AS entry, SUM(CAST(estabs_exit AS DOUBLE)) AS exit
                            FROM s
                            GROUP BY year
                            ORDER BY year""").df()
print(entry_exit)

    year     entry      exit
0   1978  642435.0  472400.0
1   1979  616145.0  467798.0
2   1980  578496.0  514495.0
3   1981  585846.0  522942.0
4   1982  574721.0  571058.0
5   1983  623924.0  528918.0
6   1984  698503.0  508969.0
7   1985  681927.0  600541.0
8   1986  710329.0  583398.0
9   1987  739325.0  627875.0
10  1988  738045.0  610497.0
11  1989  722970.0  612027.0
12  1990  698845.0  602172.0
13  1991  686812.0  648374.0
14  1992  647935.0  632986.0
15  1993  648272.0  580550.0
16  1994  680424.0  579830.0
17  1995  704648.0  573788.0
18  1996  699751.0  598546.0
19  1997  733759.0  661285.0
20  1998  699435.0  610568.0
21  1999  686195.0  629087.0
22  2000  676440.0  626495.0
23  2001  690079.0  659462.0
24  2002  755668.0  715102.0
25  2003  730983.0  663782.0
26  2004  745170.0  641399.0
27  2005  763729.0  659223.0
28  2006  839136.0  678192.0
29  2007  804827.0  725136.0
30  2008  705539.0  748850.0
31  2009  627995.0  789940.0
32  2010  632043.0  704921.0
33  2011  6555

In [29]:
firm_death=duckdb.sql("""SELECT year, SUM(CAST(estabs_entry AS DOUBLE)) AS entry, SUM(CAST(estabs_exit AS DOUBLE)) AS exit
                            FROM s
                            GROUP BY year
                            ORDER BY year""").df()
print(firm_death)

    year     entry      exit
0   1978  642435.0  472400.0
1   1979  616145.0  467798.0
2   1980  578496.0  514495.0
3   1981  585846.0  522942.0
4   1982  574721.0  571058.0
5   1983  623924.0  528918.0
6   1984  698503.0  508969.0
7   1985  681927.0  600541.0
8   1986  710329.0  583398.0
9   1987  739325.0  627875.0
10  1988  738045.0  610497.0
11  1989  722970.0  612027.0
12  1990  698845.0  602172.0
13  1991  686812.0  648374.0
14  1992  647935.0  632986.0
15  1993  648272.0  580550.0
16  1994  680424.0  579830.0
17  1995  704648.0  573788.0
18  1996  699751.0  598546.0
19  1997  733759.0  661285.0
20  1998  699435.0  610568.0
21  1999  686195.0  629087.0
22  2000  676440.0  626495.0
23  2001  690079.0  659462.0
24  2002  755668.0  715102.0
25  2003  730983.0  663782.0
26  2004  745170.0  641399.0
27  2005  763729.0  659223.0
28  2006  839136.0  678192.0
29  2007  804827.0  725136.0
30  2008  705539.0  748850.0
31  2009  627995.0  789940.0
32  2010  632043.0  704921.0
33  2011  6555

In [16]:
firms_sector=duckdb.sql("""SELECT sector, year, SUM(CAST(firms AS DOUBLE)) AS total_firms
                            FROM s
                            GROUP BY sector, year
                            ORDER BY sector, year""").df()
print(firms_sector)

    sector  year  total_firms
0       11  1978      20436.0
1       11  1979      20057.0
2       11  1980      19996.0
3       11  1981      19192.0
4       11  1982      17966.0
..     ...   ...          ...
869     81  2019     648603.0
870     81  2020     646167.0
871     81  2021     644038.0
872     81  2022     661790.0
873     81  2023     670549.0

[874 rows x 3 columns]


In [36]:
firms_age=duckdb.sql("""SELECT fage, year, SUM(CAST(firms AS DOUBLE)) AS total_firms
                            FROM s
                            GROUP BY year, fage
                            ORDER BY year, fage""").df()
firms_age=firms_age.dropna(subset=['total_firms'])
firms_age=firms_age.reset_index(drop=True)
print(firms_age)

                 fage  year  total_firms
0                a) 0  1978     488891.0
1                b) 1  1978     357355.0
2    l) Left Censored  1978    2794314.0
3                a) 0  1979     473987.0
4                b) 1  1979     364531.0
..                ...   ...          ...
462       h) 11 to 15  2023     612189.0
463       i) 16 to 20  2023     525680.0
464       j) 21 to 25  2023     371143.0
465            k) 26+  2023     881523.0
466  l) Left Censored  2023     330240.0

[467 rows x 3 columns]


In [42]:
firms_job=duckdb.sql("""SELECT year, SUM(CAST(estabs_entry AS DOUBLE) - CAST(estabs_exit AS DOUBLE)) AS net_firm_entry, SUM(CAST(job_creation AS DOUBLE)-CAST(job_destruction AS DOUBLE)) AS net_jobs
                            FROM s
                            GROUP BY year
                            ORDER BY year""").df()
print(firms_job)

    year  net_firm_entry   net_jobs
0   1978       -319983.0  2640001.0
1   1979       -297330.0  1855839.0
2   1980       -352940.0 -1837849.0
3   1981       -368070.0 -1655321.0
4   1982       -421256.0 -2997160.0
5   1983       -349627.0 -3445531.0
6   1984       -331531.0  1968101.0
7   1985       -428428.0  -110821.0
8   1986       -372783.0  -497009.0
9   1987       -409749.0 -1746227.0
10  1988       -367257.0  -657924.0
11  1989       -394569.0  -109533.0
12  1990       -381517.0  -757752.0
13  1991       -411930.0 -4759120.0
14  1992       -414949.0 -2413376.0
15  1993       -391164.0  -929014.0
16  1994       -373335.0  -450948.0
17  1995       -360532.0   665562.0
18  1996       -371834.0  -990714.0
19  1997       -414550.0   -60807.0
20  1998       -345979.0   502731.0
21  1999       -368116.0  -456746.0
22  2000       -366853.0   873727.0
23  2001       -397601.0 -2260353.0
24  2002       -394792.0 -5747035.0
25  2003       -378133.0 -1916136.0
26  2004       -377989.0 -12